<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/project/bi_projekt_przyklad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wstęp

Na podstawie opisu zbiorów oraz zmiennych nie jestem w 100% pewien czy są Państwo pewni jak ma wyglądać projekt zaliczeniowy. 

W związku z tym postanowiłem przygotować krótki przykład jak ma wyglądać projekt. Szczegółowy opis znajdą Państwo na moodle, natomiast poniżej opisuję najważniejsze kroki:

1. Wybrany zbiór danych traktujemy jako populację, dla której znamy rozkłady cech $X$ oraz $Y$.
2. Wybieramy zmienne $X$ oraz $Y$ na potrzeby badania, zmiennych $X$ kilka i mieć różny charakter (jakościowe lub ilościowe). $Y$ powinna być jedna lub maksymalnie dwie. Sugeruję aby to była zmienna ilościowa, wyliczeniowa (rozkład Poissona) czy binarna.
3. Następnie w zależności od metod(y) generujemy z populacji próbę nielosową (można na dwa sposoby, wyciągając np. próbę 10% obserwacji z górnej lub dolnej cześci rozkładu albo według modelu regresji logistycznej) i próbę losową (o ile jest potrzebna).
4. Przeprowadzamy to losowanie $B$ razy, gdzie $B$ to liczba powtórzeń
5. W każdym powtórzeniu stosujemy metodę/metody korekcji błędów dla prób nielosowych i sprawdzamy jak dobra jest dana metoda.

## Opis przykładowego zadania

W poniższym przykładzie skorzystam ze zbioru `diamonds` z pakietu `ggplot2`, który będę trakował jako populację. 

Moją zmienną celu będzie cena (`price`), a zmienymi objaśniającymi cięcie (`cut`) oraz kolor (`color`). Wybrałem dwie zmienne jakosciowe bo chcę zastosować post-stratyfikację z pakietu `survey`.

Symulacja wygląda następująco:

1. określam cenę dla całej populacji (wartość prawdziwa),
2. generuję 100 razy próbę nielosową według następujacego schematu:
    + probę o wielkosci 20% proporcjonalnie do ceny (im wyższa cena tym większe prawdopodobieństwo znalezienia się w próbie),
3. w każdym przebiegu liczę dwa estymatory:
    + naiwny -- bez żadnej korekty, zwykła średnia arytmentyczna,
    + post-stratyfikacyjny -- według dwóch zmiennych określonych powyżej (`cut`, `color`),

4. w każdym przebiegu zapisuję oszacowanie średniej ceny,
5. po wykonaniu symulacji wyliczam:
    + wartość oczekiwaną dla każdego estymatora,
    + wariancję dla każdego estymatora,
    + obciążenie dla każdego estymatora.

Oczywiście w projekcie muszą Państwo opisać jak dobre są zmienne $X$ (korelacje) czy model, który chcecie zastosować (np. `R2`, `pseudoR2` dla regresji logistycznej) oraz zinterpretować wyniki. 

Ja w tym przykładzie wykonam tylko ćwiczenia aby pokazać jak należy do projektu podejść. **TO NIE JEST PROJEKT, KTÓRY DA PAŃSTWU ZALICZENIE** to tylko przykład. 

In [ ]:
install.packages("survey")

In [12]:
library(ggplot2)
library(survey)
data(diamonds)
## sortujemy dane według ceny
diamonds <- diamonds[order(-diamonds$price),]
head(diamonds)

carat,cut,color,clarity,depth,table,price,x,y,z
<dbl>,<ord>,<ord>,<ord>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
2.29,Premium,I,VS2,60.8,60,18823,8.50,8.47,5.16
2.00,Very Good,G,SI1,63.5,56,18818,7.90,7.97,5.04
1.51,Ideal,G,IF,61.7,55,18806,7.37,7.41,4.56
2.07,Ideal,G,SI2,62.5,55,18804,8.20,8.13,5.11
2.00,Very Good,H,SI1,62.8,57,18803,7.95,8.00,5.01
2.29,Premium,I,SI1,61.8,59,18797,8.52,8.45,5.24


Wartość prawdziwa

In [6]:
cena_prawdziwa <- mean(diamonds$price)
cena_prawdziwa

[1] 3932.8

Symulacja


In [33]:
B <- 100
wyniki_naiwny <- numeric(B)
wyniki_ps <- numeric(B)
total_rows <- nrow(diamonds)

## wartości globalne
pop_cut_color <- xtabs(~ cut + color, data = diamonds)

for (b in 1:B) {
  set.seed(b) ## pełna replikacja, a poniżej losowanie próby
  nielos_id <- sample(x = 1:total_rows, size = 0.2*total_rows, 
                      prob = diamonds$price/sum(diamonds$price))
  ## wybieram rekordy
  nielosowa <- diamonds[nielos_id, ]
  ## wyliczam estymator naiwny
  wyniki_naiwny[b] <- mean(nielosowa$price)
  ## przygotowanie do post-stratyfikcji
  svydes <- svydesign(ids = ~1, data = nielosowa, weights = ~1)
  ## post-stratyfikacja
  svydes_post <- postStratify(svydes, ~ cut + color, pop_cut_color)
  ## wyliczam estymator post-stratyfikacyjny
  wyniki_ps[b] <- svymean(~ price, svydes_post)[1]
}


Wyliczam: 1) wartość oczekiwaną, 2) wariancję lub odchylenie standardowe oraz 3) obciążenie dla każdego estymatora.

In [38]:
wyniki <- data.frame(naive_oczekiwana = mean(wyniki_naiwny), 
                     naive_odchyl = sd(wyniki_naiwny), 
                     naive_obciazenie = mean(wyniki_naiwny) - cena_prawdziwa,
                     post_oczekiwana = mean(wyniki_ps), 
                     post_odchyl = sd(wyniki_ps), 
                     post_obciazenie = mean(wyniki_ps) - cena_prawdziwa,
                     cena_prawdziwa = cena_prawdziwa
)
wyniki

naive_oczekiwana,naive_odchyl,naive_obciazenie,post_oczekiwana,post_odchyl,post_obciazenie,cena_prawdziwa
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7256.763,32.38995,3323.963,7085.002,32.18905,3152.202,3932.8


Krótki komentarz:
  1. odchylenie standardowe jest takie same,
  2. obciążenie jest większe dla estymatora naiwnego niż dla post-stratyfikacyjnego ale się niewiele różni,
  3. słaby wynik dla estymatora PS jest spowodowany słabymi zmiennymi wykorzystanymi do post-stratyfikacji.